<a href="https://colab.research.google.com/github/silverstar0727/pose-estimation/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.19.3
!pip install opencv-python==4.5.1.48
!pip install tqdm==4.56.0

!pip install mediapipe==0.8.3

In [ ]:
import csv
import cv2
import numpy as np
import os
import sys
import tqdm
import pandas as pd

from mediapipe.python.solutions import pose as mp_pose

import pickle
from sklearn.externals import joblib
from sklearn.neighbors import KNeighborsClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
def landmarks(input_frame):
    input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)

      # Initialize fresh pose tracker and run it.
    with mp_pose.Pose(upper_body_only=True) as pose_tracker:
        result = pose_tracker.process(image=input_frame)
        pose_landmarks = result.pose_landmarks
      
      # Save landmarks.
    if pose_landmarks is not None:
        # Check the number of landmarks and take pose landmarks.
        assert len(pose_landmarks.landmark) == 25, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
        pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

        # Map pose landmarks from [0, 1] range to absolute coordinates to get
        # correct aspect ratio.
        frame_height, frame_width = input_frame.shape[:2]
        pose_landmarks *= np.array([frame_width, frame_height, frame_width])

        # Write pose sample to CSV.
        pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(np.float64).tolist()

        return pose_landmarks

In [ ]:
images_in_folder = 'study'
images_out_folder = 'stduy_out'
csv_out_path = 'study_result.csv'

with open(csv_out_path, 'w') as csv_out_file:
    csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)

    # Folder names are used as pose class names.
    pose_class_names = sorted([n for n in os.listdir(images_in_folder) if not n.startswith('.')])

    for pose_class_name in pose_class_names:
        print('Bootstrapping ', pose_class_name, file=sys.stderr)

        if not os.path.exists(os.path.join(images_out_folder, pose_class_name)):
            os.makedirs(os.path.join(images_out_folder, pose_class_name))

        image_names = sorted([
            n for n in os.listdir(os.path.join(images_in_folder, pose_class_name))
            if not n.startswith('.')])
        for image_name in tqdm.tqdm(image_names, position=0):
            # Load image.
            input_frame = cv2.imread(os.path.join(images_in_folder, pose_class_name, image_name))
            pose_landmarks = landmarks(input_frame)

            try:
                csv_out_writer.writerow([image_name, pose_class_name] + pose_landmarks)
            except:
                pass

Bootstrapping  do
100%|██████████| 183/183 [00:21<00:00,  8.57it/s]
Bootstrapping  do_not
100%|██████████| 162/162 [00:19<00:00,  8.16it/s]


In [ ]:
def knn(result_csv_file="study_result.csv"):
    data = pd.read_csv(result_csv_file, header=None)
    x_train, y_train = data.iloc[:, 2:], data.iloc[:, 1]

    classifier = KNeighborsClassifier(n_neighbors = 3)

    classifier.fit(x_train, y_train)
    return classifier

classifier = knn()

In [ ]:
do = sorted([n for n in os.listdir("study/do_not") if not n.startswith('.')])
do_not = sorted([n for n in os.listdir("study/do_not") if not n.startswith('.')])

for i in do_not:
    input_frame = cv2.imread(f"study/do_not/{i}")
    try:
        result = landmarks(input_frame)
        result = np.array(result).reshape(1, -1)
        print(classifier.predict(result), i)
    except:
        pass

['do_not'] image_011.jpg
['do_not'] image_012.jpg
['do_not'] image_014.jpg
['do_not'] image_015.jpg
['do_not'] image_016.jpg
['do_not'] image_017.jpg
['do_not'] image_018.jpg
['do_not'] image_019.jpg
['do_not'] image_027.jpg
['do_not'] image_028.jpg
['do_not'] image_029.jpg
['do_not'] image_030.jpg
['do_not'] image_031.jpg
['do_not'] image_032.jpg
['do_not'] image_033.jpg
['do_not'] image_034.jpg
['do_not'] image_035.jpg
['do_not'] image_036.jpg
['do_not'] image_056.jpg
['do_not'] image_057.jpg
['do_not'] image_058.jpg
['do_not'] image_059.jpg
['do_not'] image_060.jpg
['do_not'] image_061.jpg
['do_not'] image_062.jpg
['do_not'] image_063.jpg
['do_not'] image_064.jpg
['do_not'] image_065.jpg
['do_not'] image_066.jpg
['do_not'] image_067.jpg
['do_not'] image_068.jpg
['do_not'] image_069.jpg
['do_not'] image_070.jpg
['do_not'] image_071.jpg
['do_not'] image_072.jpg
['do_not'] image_073.jpg
['do_not'] image_074.jpg
['do_not'] image_075.jpg
['do_not'] image_076.jpg
['do_not'] image_077.jpg


In [ ]:
joblib.dump(classifier, 'model.pkl') 

#clf_from_joblib = joblib.load('model.pkl') 
#clf_from_joblib.predict(x_train)

['model.pkl']